<a href="https://colab.research.google.com/github/niteshsoni30/Kafka-Project/blob/main/KAFKA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Send a message to confluent kafka topic via Python

In [ ]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.1 MB/s eta 0:00:00


In [ ]:
from confluent_kafka import Producer
import json
import time

In [ ]:
import pandas as pd

df = pd.read_csv("/content/first_100_customers.csv")


In [ ]:
df.head()

,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


In [ ]:
# Converting DataFrame to Json fromat

json_records = df.to_dict(orient = "records")

json_file = "customer.json"

with open(json_file ,"w") as file:
  json.dump(json_records,file,indent =4)

print("file records to json")

file records to json


In [ ]:
# Required connection configs for Kafka producer, consumer, and admin
# bootstrap.servers=pkc-619z3.us-east1.gcp.confluent.cloud:9092
# security.protocol=SASL_SSL
# sasl.mechanisms=PLAIN
# sasl.username=CPUUEECKBRJIFO27
# sasl.password=9bcUxeNfnbl4ujwLoPHdTSa3AsVvdKdSQZ6zESsoj/HttL5e12hzzxpP2JKtleAS

# Best practice for higher availability in librdkafka clients prior to 1.7
# session.timeout.ms=45000

# client.id=ccloud-python-client-9df31087-19cf-428d-a689-b4a1decbef6c


In [ ]:
from confluent_kafka import Producer

conf = {

  "bootstrap.servers":"pkc-619z3.us-east1.gcp.confluent.cloud:9092",
  "security.protocol":"SASL_SSL",
  "sasl.mechanisms":"PLAIN",
  "sasl.username":"CPUUEECKBRJIFO27",
  "sasl.password":"9bcUxeNfnbl4ujwLoPHdTSa3AsVvdKdSQZ6zESsoj/HttL5e12hzzxpP2JKtleAS",

  "session.timeout.ms":45000,

  "client.id":"ccloud-python-client-9df31087-19cf-428d-a689-b4a1decbef6c",

}

In [ ]:
producer = Producer(conf)

In [ ]:
topic = "ecommerce"

with open("customer.json","r") as file:
  customers_data = json.load(file)

value = customers_data[0]

In [ ]:
value

{'customer_id': 0,
 'name': 'Customer_0',
 'city': 'Pune',
 'state': 'Maharashtra',
 'country': 'India',
 'registration_date': '2023-06-29',
 'is_active': False}

In [ ]:
key =value["customer_id"]
print(key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [ ]:
producer.produce(topic, key=str(key).encode("utf-8"), value= str(value).encode("utf-8"))

In [ ]:
# Sending multiple messages to Kafka

topic = "ecommerce"

def delivery_status(err,msg):
  if(err):
    print(f"message delivery fail: {err}")
  else:
    print(f"messgae delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

for record in customers_data:
  try:

    message_value = json.dumps(record)
    message_key = str(record["customer_id"]).encode("utf-8")

    producer.produce(topic, key= message_key, value= message_value , callback = delivery_status)
    producer.poll(1)

  except Exception as e:
    print(f"error message: {e}")

producer.flush()

print("Message Send To Kafka Secessfully")

messgae delivered to ecommerce [2] at offset 28
messgae delivered to ecommerce [2] at offset 29
messgae delivered to ecommerce [1] at offset 37
messgae delivered to ecommerce [1] at offset 38
messgae delivered to ecommerce [1] at offset 39
messgae delivered to ecommerce [1] at offset 40
messgae delivered to ecommerce [1] at offset 41
messgae delivered to ecommerce [0] at offset 41
messgae delivered to ecommerce [2] at offset 30
messgae delivered to ecommerce [0] at offset 42
messgae delivered to ecommerce [0] at offset 43
messgae delivered to ecommerce [0] at offset 44
messgae delivered to ecommerce [0] at offset 45
messgae delivered to ecommerce [2] at offset 31
messgae delivered to ecommerce [0] at offset 46
messgae delivered to ecommerce [1] at offset 42
messgae delivered to ecommerce [0] at offset 47
messgae delivered to ecommerce [2] at offset 32
messgae delivered to ecommerce [0] at offset 48
messgae delivered to ecommerce [1] at offset 43
messgae delivered to ecommerce [0] at of

In [ ]:
# Read Message From Kafka

from confluent_kafka import Consumer ,KafkaError, KafkaException

conf = {

  "bootstrap.servers":"pkc-619z3.us-east1.gcp.confluent.cloud:9092",
  "security.protocol":"SASL_SSL",
  "sasl.mechanisms":"PLAIN",
  "sasl.username":"CPUUEECKBRJIFO27",
  "sasl.password":"9bcUxeNfnbl4ujwLoPHdTSa3AsVvdKdSQZ6zESsoj/HttL5e12hzzxpP2JKtleAS",

  "session.timeout.ms":45000,

  "client.id":"ccloud-python-client-9df31087-19cf-428d-a689-b4a1decbef6c",

   "auto.offset.reset":"earliest",
   "group.id":"customer_group",
}


In [ ]:
consumer = Consumer(conf)

In [ ]:
consumer

In [ ]:
topic = "ecommerce"
consumer.subscribe([topic])


def process_message(message):
  try:
    if message.error():
      if (message.error.code()) == KafkaError._PARTITION_EOF:
        print("end of partition reached {0}/{1}")

      else:
        raise KafkaException(message.error())

    else:
        key = message.key().decode("utf-8")
        value = json.loads(message.value().decode("utf-8"))

        print(f"Received message : key {key}, value: {value}")

  except Exception as e:
      print(f"Error consuming/processing message : {e}")

In [ ]:
# Poll message continuously

try:
  print("Listening for messages, press ctrl + C to exit")

  while True:
    message = consumer.poll(timeout = 1.0)
    if message:
      process_message(message)

except KeyboardInterrupt:
  print("Interrupted by user, shutting down consumer")

finally:
  consumer.close()

Listening for messages, press ctrl + C to exit
Error consuming/processing message : Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Received message : key 0, value: {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}
Received message : key 1, value: {'customer_id': 1, 'name': 'Customer_1', 'city': 'Bangalore', 'state': 'Tamil Nadu', 'country': 'India', 'registration_date': '2023-12-07', 'is_active': True}
Received message : key 8, value: {'customer_id': 8, 'name': 'Customer_8', 'city': 'Pune', 'state': 'Karnataka', 'country': 'India', 'registration_date': '2023-06-22', 'is_active': True}
Received message : key 13, value: {'customer_id': 13, 'name': 'Customer_13', 'city': 'Chennai', 'state': 'Karnataka', 'country': 'India', 'registration_date': '2023-11-06', 'is_active': True}
Received message : key 17, value: {'customer_id': 17, 'name': 'Customer_17', 'cit